# Day 8

The expedition comes across a peculiar patch of tall trees all planted carefully in a grid. The Elves explain that a previous expedition planted these trees as a reforestation effort. Now, they're curious if this would be a good location for a tree house.

First, determine whether there is enough tree cover here to keep a tree house hidden. To do this, you need to count the number of trees that are visible from outside the grid when looking directly along a row or column.

The Elves have already launched a quadcopter to generate a map with the height of each tree (your puzzle input). For example:
```
30373
25512
65332
33549
35390
```
Each tree is represented as a single digit whose value is its height, where 0 is the shortest and 9 is the tallest.

A tree is visible if all of the other trees between it and an edge of the grid are shorter than it. Only consider trees in the same row or column; that is, only look up, down, left, or right from any given tree.

All of the trees around the edge of the grid are visible - since they are already on the edge, there are no trees to block the view. In this example, that only leaves the interior nine trees to consider:

  - The top-left 5 is visible from the left and top. (It isn't visible from the right or bottom since other trees of height 5 are in the way.)
  - The top-middle 5 is visible from the top and right.
  - The top-right 1 is not visible from any direction; for it to be visible, there would need to only be trees of height 0 between it and an edge.
  - The left-middle 5 is visible, but only from the right.
  - The center 3 is not visible from any direction; for it to be visible, there would need to be only trees of at most height 2 between it and an edge.
  - The right-middle 3 is visible from the right.
  - In the bottom row, the middle 5 is visible, but the 3 and 4 are not.
  
With 16 trees visible on the edge and another 5 visible in the interior, a total of 21 trees are visible in this arrangement.

Consider your map; how many trees are visible from outside the grid?

In [1]:
from read_data import get_data
import numpy as np
import re
trees = get_data(8)
# log


In [2]:
trees

['213301311230020143114320444110403320013225525311631030063110421431345121304222223004301412232100001',
 '032203111100043323010213335301511304130605406233153314443635022154110215552041255340234040130323222',
 '211033104233224304235521203100252423434120553365402362266533500152442121543412024410024400242202302',
 '013233443223404100042450150455321226633161540105245332600255253210530355343333525022032244210410030',
 '100023412423313250410335224414340304541343205513226532163144253316101245551432014201003142341223120',
 '232241013411335302431133421201615210501105064501562020244216516152310063135512221455045300342211403',
 '001211403144420455554423216332215200446124050665565420055553546105000034160513122231050524220140011',
 '010222034210203122402014343566262222246103065613373733360061263604223104106135234020024543242111403',
 '133113440330510405504554625242222200403366266153277146434533626152126510506334524322514123130210430',
 '10111304143200141014251151352650300253216217143623367347552536

In [128]:
def get_trees_near(trees, i, j, n_cols, n_rows):
    return {
        "above": [trees[x][i] for x in range(0, j)][::-1],
        "below": [trees[x][i] for x in range(j+1, n_cols)],
        "left": [trees[j][x] for x in range(0, i)][::-1],
        "right": [trees[j][x] for x in range(i+1, n_rows)]
    }

In [129]:
def is_invisible(tree_height, trees_near):
        return all(tree_height <= max(v) for v in trees_near.values())

In [134]:

def count_visible(trees):
    n_rows = len(trees)
    n_cols = len(trees[0])
    count = 2*n_cols + 2*n_rows - 4

    for j in range(1, n_rows-1):
        for i in range (1, n_cols-1):
            tree_height = trees[j][i]
            trees_near = get_trees_near(trees, i, j, n_cols, n_rows)
            if not is_invisible(tree_height, trees_near):
                count += 1

    return count

In [135]:
count_visible(trees)

1679

Content with the amount of tree cover available, the Elves just need to know the best spot to build their tree house: they would like to be able to see a lot of trees.

To measure the viewing distance from a given tree, look up, down, left, and right from that tree; stop if you reach an edge or at the first tree that is the same height or taller than the tree under consideration. (If a tree is right on the edge, at least one of its viewing distances will be zero.)

The Elves don't care about distant trees taller than those found by the rules above; the proposed tree house has large eaves to keep it dry, so they wouldn't be able to see higher than the tree house anyway.

In the example above, consider the middle 5 in the second row:

```
30373
25512
65332
33549
35390
```

  - Looking up, its view is not blocked; it can see 1 tree (of height 3).
  - Looking left, its view is blocked immediately; it can see only 1 tree (of height 5, right next to it).
  - Looking right, its view is not blocked; it can see 2 trees.
  - Looking down, its view is blocked eventually; it can see 2 trees (one of height 3, then the tree of height 5 that blocks its view).

A tree's scenic score is found by multiplying together its viewing distance in each of the four directions. For this tree, this is 4 (found by multiplying 1 * 1 * 2 * 2).

However, you can do even better: consider the tree of height 5 in the middle of the fourth row:

```
30373
25512
65332
33549
35390
```

  - Looking up, its view is blocked at 2 trees (by another tree with a height of 5).
  - Looking left, its view is not blocked; it can see 2 trees.
  - Looking down, its view is also not blocked; it can see 1 tree.
  - Looking right, its view is blocked at 2 trees (by a massive tree of height 9).

This tree's scenic score is 8 (2 * 2 * 1 * 2); this is the ideal spot for the tree house.

Consider each tree on your map. What is the highest scenic score possible for any tree?

In [136]:
def calculate_score(tree_height, trees_near):
    scores = {
        "above": 0,
        "below": 0,
        "left": 0,
        "right": 0
    }

    for direction, v in trees_near.items():
        for tree in v:
            scores[direction] += 1
            if tree >= tree_height:
                break
    
    return np.product([v for v in scores.values()])

In [137]:
def max_score(trees):
    n_rows = len(trees)
    n_cols = len(trees[0])
    score = 0

    for j in range(1, n_rows-1):
        for i in range (1, n_cols-1):
            tree_height = trees[j][i]
            trees_near = get_trees_near(trees, i, j, n_cols, n_rows)
            tree_score = calculate_score(tree_height, trees_near)
            if tree_score > score:
                score = tree_score

    return score

In [138]:
max_score(trees)

536625